#### Visualize data fro mongodb on quick-chart 
This notebook pulls data set from [MongoDB Atlas -- The Cloud-Native Database](https://www.mongodb.com/cloud/atlas/lp/try4?utm_source=google&utm_campaign=search_gs_pl_evergreen_atlas_core-high-int_prosp-brand_gic-null_emea-ie_ps-all_desktop_eng_lead&utm_term=mongodb%20atlas&utm_medium=cpc_paid_search&utm_ad=e&utm_ad_campaign_id=19630910055&adgroup=145923638859&cq_cmp=19630910055&gad=1&gclid=Cj0KCQjwsIejBhDOARIsANYqkD2ewYCphoJvBv_op03rwncSNye4Mq_RihH9EaZKsnFK4BV-roOCOrYaAuSREALw_wcB) database and then uses [QuickChart](https://quickchart.io/) library to visualize data
- create connection with Mongodb Atlas
- visualize data from mongodb using QuickChart 

In [ ]:
!pip install quickchart.io

In [ ]:
from quickchart import QuickChart
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
from pymongo import MongoClient
import os
import logging
import sys
import re

In [ ]:
#database_username = 'database_username'
database_username = os.environ.get('database_username', 'HERE_GOES_YOUR_DATABASE_USERNAME')

# database password
password =  os.environ.get('password','HERE_GOES_YOUR_DATABASE_PASSWORD')

# cluster url
cluster_url = os.environ.get('cluster_url',"HERE_GOES_YOUR_CLUSTER_URL")

# database name
database = os.environ.get('database','HERE_GOES_YOUR_DATABASE_NAME')

# database collection
collection = os.environ.get('collection','HERE_GOES_YOUR_COLLECTION_NAME')

# Input parameters for chart
# chart type
chart_type = os.environ.get('chart_type','HERE_GOES_YOUR_CHART_TYPE')

# chart data    
chart_data = os.environ.get('chart_data','HERE_GOES_YOUR_CHART_DATA')

# chart labels
chart_labels = os.environ.get('chart_labels','HERE_GOES_YOUR_CHART_LABELS')

# chart title
chart_title = os.environ.get('chart_title',"HERE_GOES_YOUR_CHART_TITLE")

#chart height
chart_height = os.environ.get('chart_height','HERE_GOES_YOUR_CHART_HEIGHT')

#chart width
chart_width = os.environ.get('chart_width','HERE_GOES_YOUR_CHART_WIDTH')

# chart data
chart_datasets = os.environ.get('chart_datasets','HERE_GOES_YOUR_CHART_DATASETS')

# path and file name for output
output_data_image = os.environ.get('output_data_image', 'HERE_GOES_YOUR_OUTPUT_DATA_IMAGE_JPG')


In [ ]:
parameters = list(
    map(lambda s: re.sub('$', '"', s),
        map(
            lambda s: s.replace('=', '="'),
            filter(
                lambda s: s.find('=') > -1 and bool(re.match(r'[A-Za-z0-9_]*=[.\/A-Za-z0-9]*', s)),
                sys.argv
            )
    )))

for parameter in parameters:
    logging.warning('Parameter: ' + parameter)
    exec(parameter)

In [ ]:
# MongoDB connection settings
# Connection URL. This is where your mongodb server is running.
url = f"mongodb+srv://{database_username}:{password}@{cluster_url}/test?retryWrites=true&w=majority"

# Connect to MongoDB
client = MongoClient(url)

In [ ]:
# MongoDB connection testing
# Access specific database
db = client[database]

# Access specific collection
collection = db[collection]

# Retrieve all weather data
#documents = collection.find()

# Or retrieve 100 documents
documents = collection.find().limit(100)

In [ ]:
# Initialize empty lists for data extraction
temperatures = []
pressures = []
wind_speeds = []

In [ ]:
# Extract relevant weather data fields
for document in documents:
    temperatures.append(document['airTemperature']['value'])
    pressures.append(document['pressure']['value'])
    wind_speeds.append(document['wind']['speed']['rate'])


In [ ]:
print("Temperatures: ", temperatures)
print("Pressures: ", pressures)
print("Wind speeds: ", wind_speeds)

In [ ]:
# Define your chart data configuration
qc = QuickChart()
qc.height = chart_height
qc.width = chart_width
qc.config = {
    'type': 'line',
    'data': {
        'labels': chart_labels,
        'datasets': chart_datasets
    },
    'options': {
        'scales': {
            'y': {
                'beginAtZero': True
            },
            'x': {
                'beginAtZero': True
            }
        }
    }
}

In [ ]:
# print the chart url
print(qc.get_short_url())

In [ ]:
# Or show the chart in this notebook
# Send a GET request to the image URL and retrieve the image
response = requests.get(qc.get_short_url())
image = Image.open(BytesIO(response.content))

# Display the image
plt.imshow(image)
plt.axis('off')
plt.show()

In [ ]:
#Save the image

# Convert the image to the 'RGB' mode if necessary
if image.mode != 'RGB':
    image = image.convert('RGB')
    
# Save the image to the specified file
image.save(output_data_image)

print("Image saved successfully!")